In [16]:
import streamlit as st
from streamlit_gsheets import GSheetsConnection
import pickle
import pandas as pd
import copy


# Establishing a Google Sheets connection
conn = st.connection("gsheets", type=GSheetsConnection)

# Fetch data dhp = data history prediction
existing_dhp = conn.read(worksheet="Data Histori Prediksi Suatu Prodi", ttl=5)
existing_dhp = existing_dhp.dropna(how="all")

# Fetch data djm = data jumlah mahasiswa
existing_djm = conn.read(worksheet="Data Jumlah Mahasiswa")
existing_djm = existing_djm.dropna(how="all")
existing_djm = existing_djm.replace('#N/A ()', 0)

# Fetch existing formulas data
existing_formula = conn.read(worksheet="Rumus Pemantauan", usecols=list(range(7)), ttl=5)
existing_formula = existing_formula.dropna(how="all")

# Dropdown options for Lembaga
lembaga_options = existing_djm['Lembaga'].unique()
formula_options = existing_formula['Nama Rumus'].unique()

unused_column = ['Kode Prodi', 'Kode Prodi UGM', 'Kode Fakultas', 'Program Studi', 'BAN PT', 'Departemen', 'Kluster', 'PDDIKTI x BAN']
djm = existing_djm.drop(unused_column, axis=1)

# st.write(djm)
# Input fields
# input_prodi = st.text_input("Masukkan Nama Program Studi : ")
input_prodi = existing_djm["Prodi"]

# CRUD Form
max_value = int(existing_djm.columns[-1])
min_value = int(existing_djm.columns[12])

input_predict_year = st.slider("Masukkan Tahun yang Ingin Diprediksi (ex: 2025) : ", min_value=min_value+1, max_value=max_value+1)
input_last_year = input_predict_year - 1

input_years_to_predict = st.slider("Masukkan Proyeksi Prediksi (Dalam Satuan Tahun) : ", min_value=1, max_value=10)

input_formula = st.radio("Formula yang Digunakan", ["Sudah Ada", "Baru"])


model = pickle.load(open(r"D:\jumapro\next_year_students_prediction.sav", "rb"))


print('existing formula:', existing_formula)
selected_formulas = {}
if input_formula == "Sudah Ada":
    for lembaga_name in lembaga_options:
        # Filter formulas by the selected Lembaga
        formula_options = existing_formula[existing_formula['Lembaga'] == lembaga_name]['Nama Rumus'].unique()

        # Dropdown to select formula for the current Lembaga
        selected_formulas[lembaga_name] = st.selectbox(f"Pilih Rumus yang Digunakan bagi Prodi di bawah Lembaga {lembaga_name} : ", formula_options)
        selected_formulas_name = existing_formula[(existing_formula['Nama Rumus'] == selected_formulas[lembaga_name]) & (existing_formula['Lembaga'] == lembaga_name)].iloc[0]
        # st.write(selected_formulas_name)

    existing_formula_copy = copy.deepcopy(existing_formula)
    existing_djm_copy = copy.deepcopy(existing_djm)
    print('selected_formulas:', selected_formulas)
    for index, row in existing_djm.iterrows():
        lembaga_prodi = row['Lembaga']
        prodi_name = row['Prodi']
        print(index, 'lembaga_prodi:', lembaga_prodi, 'prodi_name:', prodi_name)
            
        # Mengambil baris formula yang dipilih
        selected_formula = existing_formula[(existing_formula['Nama Rumus'] == selected_formulas[lembaga_prodi]) & (existing_formula['Lembaga'] == lembaga_prodi)].iloc[0]
        
        # st.write(selected_formula)
        # Cek kriteria
        input_kriteria = selected_formula["Kriteria"]

        if input_kriteria == "Persentase Penurunan":
            input_ambang_batas_persen = selected_formula["Ambang Batas (%)"]
            input_banyak_data_ts = selected_formula["Banyak Data TS"]
            input_ambang_batas_jumlah = None
            input_fields = {}
            for i in range(int(input_banyak_data_ts)):
                field_name = f"input_jumlah_mahasiswa_ts{i}"
                # input_fields[field_name] = st.number_input(f"Masukkan Jumlah Mahasiswa TS-{i}:", value=0)
                try:
                    input_fields[field_name] = existing_djm[input_predict_year-i-1]
                    # print(f'jmlh {i} | {field_name} | {input_predict_year-i} | { existing_djm[input_predict_year-i]}')
                except KeyError:
                    # MUNGKIN datanya ga cukup, misal pilih 2014, tapi datanya cmn ada dari 2013, trus ambil -3 tahun
                    input_fields[field_name] = 1
                

        else:
            input_ambang_batas_jumlah = selected_formula["Ambang Batas (Jumlah)"]
            # input_jumlah_mahasiswa_ts = None
            input_ambang_batas_persen = None
            input_fields = None
            # # Calculate predictions
            # current_students = row[input_last_year]
            # tahun_tidak_lolos = f"Lebih dari {input_years_to_predict} Tahun ke Depan"  # Default value

            # for i in range(1, input_years_to_predict + 1):
            #     next_year = input_last_year + i
            #     column_name = f'{next_year} (Prediksi)'

            #     # Lakukan prediksi menggunakan model untuk current_students dari baris ini
            #     prediksi_mahasiswa = model.predict([[current_students]])[0]
            #     existing_djm.at[index, column_name] = int(prediksi_mahasiswa)

            #     # Perbarui current_students untuk iterasi berikutnya
            #     current_students = prediksi_mahasiswa

            #     # Cek apakah jumlah mahasiswa di bawah ambang batas
            #     if prediksi_mahasiswa < input_ambang_batas_jumlah:
            #         tahun_tidak_lolos = next_year
            # print('after milih jumlah mahasiswa, sudah menghitung variable tahun_tidak_lolos: ', tahun_tidak_lolos)
            # existing_djm.at[index, f"Hasil Prediksi Pemantauan ({input_predict_year})"] = "Lolos" if prediksi_mahasiswa >= input_ambang_batas_jumlah else "Tidak Lolos"
else:
    input_kriteria = st.radio("Kriteria", ["Jumlah Mahasiswa", "Persentase Penurunan"])
    if input_kriteria == "Persentase Penurunan":
        input_ambang_batas_persen = st.slider("Ambang Batas Persentase Maksimal (%)", min_value=1, max_value=100, step=1)
        input_banyak_data_ts = st.slider("Masukkan Banyak Tahun yang Dipakai untuk Persentase Penurunan : ", min_value=2, max_value=5, step=1)
        input_ambang_batas_jumlah = None

        input_fields = {}
        for i in range(input_banyak_data_ts - 1):
            field_name = f"input_jumlah_mahasiswa_ts{i}"
            input_fields[field_name] = existing_djm[input_predict_year-i-1]
                
    else:
        input_ambang_batas_jumlah = st.number_input("Ambang Batas Jumlah Mahasiswa Minimal", min_value=1, step=1)
        input_jumlah_mahasiswa_ts = existing_djm[input_predict_year-1]
        input_ambang_batas_persen = None
        input_fields = None

model = pickle.load(open(r"D:\jumapro\next_year_students_prediction.sav", "rb"))

# existing_djm = existing_djm.dropna()
# if input_kriteria == "Jumlah Mahasiswa":
existing_djm.rename(columns={input_last_year:"current_students"}, inplace=True)
#     new_data_prodi = {
#         'Prodi': [input_prodi],
#         'current_students': [input_jumlah_mahasiswa_ts]
#     }
# else:
#     new_data_prodi = {
#         'Prodi': [input_prodi],
#         'current_students': [input_fields[list(input_fields.keys())[0]]]
#     }

# data_prodi = pd.DataFrame(new_data_prodi)
all_tahun_tidak_lolos = dict()
# Prediksi beberapa tahun ke depan
for index, row in existing_djm.iterrows():
    # Ambil data current_students dari baris saat ini
    current_students = row['current_students']
    tahun_tidak_lolos = f"Lebih dari {input_years_to_predict} Tahun ke Depan"  # Default value

    for i in range(1, input_years_to_predict + 1):
        next_year = input_last_year + i
        column_name = f'{next_year} (Prediksi)'

        # Lakukan prediksi menggunakan model untuk current_students dari baris ini
        prediksi_mahasiswa = model.predict([[current_students]])[0]
        existing_djm.at[index, column_name] = int(prediksi_mahasiswa)

        # Perbarui current_students untuk iterasi berikutnya
        current_students = prediksi_mahasiswa

        # Cek apakah jumlah mahasiswa di bawah ambang batas untuk kriteria Jumlah Mahasiswa
        if input_kriteria == "Jumlah Mahasiswa" and prediksi_mahasiswa < input_ambang_batas_jumlah:
            tahun_tidak_lolos = next_year

        # Cek apakah jumlah mahasiswa di bawah ambang batas untuk kriteria Persentase Penurunan
        elif input_kriteria == "Persentase Penurunan":
            ambang_batas_jumlah_mahasiswa = int(row["current_students"] * (1 - input_ambang_batas_persen / 100))
            if prediksi_mahasiswa < ambang_batas_jumlah_mahasiswa:
                tahun_tidak_lolos = next_year
        all_tahun_tidak_lolos[index] = tahun_tidak_lolos
        
    # Update kolom hasil pemantauan dan tahun tidak lolos
    if input_kriteria == "Jumlah Mahasiswa":
        existing_djm.at[index, f"Hasil Prediksi Pemantauan ({input_predict_year})"] = "Lolos" if prediksi_mahasiswa >= input_ambang_batas_jumlah else "Tidak Lolos"
    elif input_kriteria == "Persentase Penurunan":
        existing_djm.at[index, f"Hasil Prediksi Pemantauan ({input_predict_year})"] = "Lolos" if prediksi_mahasiswa >= ambang_batas_jumlah_mahasiswa else "Tidak Lolos"
    



data_predict_years = [f"{next_year} (Prediksi)" for next_year in range(input_predict_year+1, input_predict_year+input_years_to_predict)]
data_predict_target= [f"{input_predict_year} (Prediksi)"]

# Fungsi untuk menghitung persentase penurunan
def hitung_persentase_penurunan(data, predict_year):
    # data_mahasiswa_start_year = input_fields["input_jumlah_mahasiswa_ts0"]
    # end_year = predict_year
    # penurunan_total = (data[f"{end_year} (Prediksi)"] - data_mahasiswa_start_year)
    # persentase_penurunan = (penurunan_total / 2) * 100
    # return persentase_penurunan
    ts_1 = data[f"{predict_year} (Prediksi)"]
    ts_0 = data["current_students"]
    penurunan = (ts_0 - ts_1) / ts_1

    persentase_penurunan = penurunan * 100

    return persentase_penurunan


# Fungsi untuk menghitung persentase penurunan dengan lebih dari satu tahun data
def hitung_persentase_penurunan_lebih_dari_satu(data, predict_year, banyak_data_ts):
    total_penurunan = 0

    # Iterasi dari TS-1 hingga TS-n (n = banyak_data_ts)
    for i in range(int(banyak_data_ts) - 1):
        if i == 0:
            ts_1 = data[f"{predict_year} (Prediksi)"]
            ts_0 = data["current_students"]
        else:
            ts_1 = input_fields[f"input_jumlah_mahasiswa_ts{i-1}"]
            ts_0 = input_fields[f"input_jumlah_mahasiswa_ts{i}"]
        # print('i ts0 ts1', i, ts_0, ts_1)
       
        penurunan = (ts_0 - ts_1) / ts_1
        total_penurunan += penurunan
       

    # Hitung rata-rata penurunan
    rata_rata_penurunan = total_penurunan / (banyak_data_ts - 1)
    persentase_penurunan = rata_rata_penurunan * 100

    return persentase_penurunan

# Perbarui fungsi prediksi_dan_penilaian untuk menggunakan fungsi yang baru
# print('all tahun tidak lolos:', all_tahun_tidak_lolos)
def prediksi_dan_penilaian(input_prodi, input_predict_year, input_last_year_data, input_years_to_predict, input_kriteria, input_ambang_batas_jumlah, input_ambang_batas_persen, input_fields):
    # print('all args:')
    # print('input prodi:', type(input_prodi), input_prodi)
    args = input_predict_year, input_last_year_data, input_years_to_predict, input_kriteria, input_ambang_batas_jumlah, input_ambang_batas_persen
    # print('args:', args)
    # print('input fields:', type(input_fields), input_fields)
    with open('input_fields.pkl', 'wb+') as file:
        pickle.dump(input_fields, file)
    with open('input_prodi.pkl', 'wb') as file:
        pickle.dump(input_prodi, file)     
    # print('export arg done')   
    # print('before loop, edjm length:', existing_djm.shape, len(existing_djm))
    # print('existing djm head: ', existing_djm.head())
    for index, row in existing_djm.iterrows():
        # print('LOOP BEFORE IF', index)
        # Penilaian kelolosan berdasarkan kriteria
        if input_kriteria == "Jumlah Mahasiswa":
            # print('looping jumlah mahasiswa', index)
            hasil_prediksi_pemantauan = "Lolos" if row[f"{input_predict_year} (Prediksi)"] > input_ambang_batas_jumlah else "Tidak Lolos"
            existing_djm.at[index, f"Hasil Prediksi Pemantauan ({input_predict_year})"] = hasil_prediksi_pemantauan
            # existing_djm[f"Hasil Prediksi Pemantauan ({input_predict_year})"] = hasil_prediksi_pemantauan
            existing_djm.at[index, "Jumlah Mahasiswa Minimal"] = input_ambang_batas_jumlah
            # existing_djm["Jumlah Mahasiswa Minimal"] = input_ambang_batas_jumlah
            existing_djm.at[index,"Tahun Tidak Lolos (Prediksi)"] = str(all_tahun_tidak_lolos[index])
            # existing_djm["Tahun Tidak Lolos (Prediksi)"] = str(tahun_tidak_lolos)

            ordered_data_prodi = ["Prodi"] + ["Lembaga"] + ["current_students"] + data_predict_target + ["Jumlah Mahasiswa Minimal"] + [f"Hasil Prediksi Pemantauan ({input_predict_year})"] + data_predict_years + ["Tahun Tidak Lolos (Prediksi)"]
            tampil_data_prodi = existing_djm[ordered_data_prodi]
            tampil_data_prodi.rename(columns={'current_students': f'{input_last_year} (Saat Ini)'}, inplace=True)

            # updated_dhp = pd.concat([existing_dhp, tampil_data_prodi], ignore_index=True)
            # conn.update(worksheet="Histori Prediksi Suatu Prodi", data=updated_dhp)

        elif input_kriteria == "Persentase Penurunan":
            # print('looping persentase penurunan', index)
            if input_banyak_data_ts > 2:
                persentase_penurunan = hitung_persentase_penurunan_lebih_dari_satu(existing_djm, input_predict_year, input_banyak_data_ts)
            else:
                persentase_penurunan = hitung_persentase_penurunan(existing_djm, input_predict_year)
            
            existing_djm["Hitung Persentase Penurunan"] = persentase_penurunan
            # hasil_prediksi_pemantauan = "Lolos" if persentase_penurunan.values[0] <= input_ambang_batas_persen else "Tidak Lolos"
            hasil_prediksi_pemantauan = "Lolos" if persentase_penurunan.values[0] <= input_ambang_batas_persen else "Tidak Lolos"
            
            
            convert_percent_to_ambang_batas_jumlah_mahasiswa = int(row["current_students"] * (1 - input_ambang_batas_persen / 100))
            # ambang_batas_jumlah_mahasiswa = int(data_prodi["current_students"] * (1 - input_ambang_batas_persen / 100))
            # existing_djm["Hitung Persentase Penurunan"] = persentase_penurunan
            existing_djm.at[index, "Persentase Penurunan Maksimal"] = f"{input_ambang_batas_persen}%"
            existing_djm.at[index, "Ambang Batas Jumlah Mahasiswa Minimal"] = convert_percent_to_ambang_batas_jumlah_mahasiswa
            existing_djm.at[index, f"Hasil Prediksi Pemantauan ({input_predict_year})"] = hasil_prediksi_pemantauan
            existing_djm.at[index, "Tahun Tidak Lolos (Prediksi)"] = str(all_tahun_tidak_lolos[index])
            # print(f'row existing djm at index {index}: ', existing_djm.loc[index])
            for col, value in input_fields.items():
                if col!="input_jumlah_mahasiswa_ts0":
                    existing_djm[col] = value
            ts = [f"input_jumlah_mahasiswa_ts{i}" for i in range(1, int(input_banyak_data_ts-1))]
            ts = sorted(ts, reverse=True)

            ordered_data_prodi = ["Prodi"] + ["Lembaga"] + ts + ["current_students"] + data_predict_target + ["Persentase Penurunan Maksimal"] + ["Hitung Persentase Penurunan"] + [f"Hasil Prediksi Pemantauan ({input_predict_year})"] + ["Ambang Batas Jumlah Mahasiswa Minimal"] + data_predict_years + ["Tahun Tidak Lolos (Prediksi)"]
            tampil_data_prodi = existing_djm[ordered_data_prodi]
            tampil_data_prodi.rename(columns={'current_students': f'{input_last_year} (Saat Ini)'}, inplace=True)
            rename_ts = {f"input_jumlah_mahasiswa_ts{i+1}": f"{input_last_year-i-1}" for i in range(int(input_banyak_data_ts-1))}
            tampil_data_prodi.rename(columns=rename_ts, inplace=True)
            tampil_data_prodi.rename(columns={'current_students': f'{input_last_year_data} (Saat Ini)'}, inplace=True)

            # updated_dhp = pd.concat([existing_dhp, tampil_data_prodi], ignore_index=True)
            # conn.update(worksheet="Histori Prediksi Suatu Prodi", data=updated_dhp)
        else:
            print('Other ', index)

    return tampil_data_prodi
        # return existing_djm
    




    

2024-09-14 00:21:19.736 No runtime found, using MemoryCacheStorageManager
2024-09-14 00:21:22.607 No runtime found, using MemoryCacheStorageManager
2024-09-14 00:21:22.610 No runtime found, using MemoryCacheStorageManager
C:\Users\Anggraini D\AppData\Local\Temp\ipykernel_27328\800065126.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  existing_djm = existing_djm.replace('#N/A ()', 0)
2024-09-14 00:21:23.611 No runtime found, using MemoryCacheStorageManager
2024-09-14 00:21:23.614 No runtime found, using MemoryCacheStorageManager


existing formula:         Lembaga                     Nama Rumus              Kriteria  \
0        BAN-PT         Rumus Pertama BAN 2024  Persentase Penurunan   
1   LAM INFOKOM          Rumus Pertama Infokom  Persentase Penurunan   
2    LAM TEKNIK              Rumus Pertama Tek  Persentase Penurunan   
3     LAM-PTKes              Rumus Pertama Kes  Persentase Penurunan   
4       LAMEMBA             Rumus Pertama EMBA  Persentase Penurunan   
5       LAMSAMA             Rumus Pertama SAMA  Persentase Penurunan   
6        BAN-PT                Rumus Kedua BAN      Jumlah Mahasiswa   
7   LAM INFOKOM            Rumus Kedua Infokom      Jumlah Mahasiswa   
8    LAM TEKNIK                Rumus Kedua Tek      Jumlah Mahasiswa   
9     LAM-PTKes                Rumus Kedua Kes      Jumlah Mahasiswa   
10      LAMEMBA               Rumus Kedua Emba      Jumlah Mahasiswa   
11      LAMSAMA               Rumus Kedua SAMA      Jumlah Mahasiswa   
12       BAN-PT              update bugfix BAN

d:\python\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.3.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\python\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.3.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [15]:
lembaga_prodi = 'LAMEMBA'
prodi_name = 'Magister Manajemen Pendidikan Tinggi'
print(selected_formulas)
existing_formula_copy[(existing_formula_copy['Nama Rumus'] == selected_formulas[lembaga_prodi]) & (existing_formula_copy['Lembaga'] == lembaga_prodi)]

{'BAN-PT': 'Rumus Pertama BAN 2024', 'LAM-PTKes': 'Rumus Pertama Kes', 'LAM TEKNIK': 'Rumus Pertama Tek', 'LAMSAMA': 'Rumus Pertama SAMA', 'LAM INFOKOM': 'Rumus Pertama Infokom', 'LAMEMBA': 'Rumus Pertama EMBA'}


,Lembaga,Nama Rumus,Kriteria,Banyak Data TS,Ambang Batas (%),Ambang Batas (Jumlah),Tanggal Mulai Berlaku
4,LAMEMBA,Rumus Pertama EMBA,Persentase Penurunan,4.0,25.0,NaN,NaN


In [19]:
existing_djm_copy[existing_djm_copy.columns[5:15]]

,Program Studi,BAN PT,Departemen,Kluster,PDDIKTI x BAN,Jenjang,Lembaga,2013,2014,2015
0,Doktor Ilmu Kedokteran dan Kesehatan,ILMU KEDOKTERAN DAN KESEHATAN,NaN,Medika,0.0,S-3,BAN-PT,53,38,37
1,Magister Ilmu Biomedik,ILMU BIOMEDIK,NaN,Medika,0.0,S-2,LAM-PTKes,49,51,44
2,Magister Ilmu Kedokteran Klinis,ILMU KEDOKTERAN KLINIS,NaN,Medika,0.0,S-2,LAM-PTKes,75,96,82
3,Magister Ilmu Kedokteran Tropis,ILMU KEDOKTERAN TROPIS,NaN,Medika,0.0,S-2,LAM-PTKes,14,17,11
4,Magister Ilmu Pendidikan Kedokteran dan Kesehatan,ILMU PENDIDIKAN KEDOKTERAN DAN KESEHATAN,NaN,Medika,0.0,S-2,LAM-PTKes,5,13,12
...,...,...,...,...,...,...,...,...,...,...
143,Magister Pengkajian Amerika,Magister Pengkajian Amerika,NaN,Soshum,1.0,S-2,BAN-PT,24,18,9
144,Kajian Budaya Timur Tengah,Magister Kajian Budaya Timur Tengah,NaN,Soshum,1.0,S-2,BAN-PT,0,0,0
145,Magister Kajian Budaya dan Media,Magister Kajian Budaya Dan Media,NaN,Pascasarjana,1.0,S-2,BAN-PT,41,29,30
146,Magister Ketahanan Nasional,Magister Ketahanan Nasional,NaN,Pascasarjana,1.0,S-2,BAN-PT,70,57,40


In [12]:
prediksi_dan_penilaian(input_prodi, input_predict_year, input_last_year, input_years_to_predict, input_kriteria, input_ambang_batas_jumlah, input_ambang_batas_persen, input_fields)


C:\Users\Anggraini D\AppData\Local\Temp\ipykernel_27328\1547991785.py:296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tampil_data_prodi.rename(columns={'current_students': f'{input_last_year} (Saat Ini)'}, inplace=True)
C:\Users\Anggraini D\AppData\Local\Temp\ipykernel_27328\1547991785.py:298: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tampil_data_prodi.rename(columns=rename_ts, inplace=True)
C:\Users\Anggraini D\AppData\Local\Temp\ipykernel_27328\1547991785.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyda

,Prodi,Lembaga,2011,2012,2013 (Saat Ini),2014 (Prediksi),Persentase Penurunan Maksimal,Hitung Persentase Penurunan,Hasil Prediksi Pemantauan (2014),Ambang Batas Jumlah Mahasiswa Minimal,Tahun Tidak Lolos (Prediksi)
0,Doktor Ilmu Kedokteran dan Kesehatan,BAN-PT,1,1,53,51.0,20.0%,-31.397213,Lolos,42.0,Lebih dari 1 Tahun ke Depan
1,Magister Ilmu Biomedik,LAM-PTKes,1,1,49,48.0,20.0%,-31.958617,Lolos,39.0,Lebih dari 1 Tahun ke Depan
2,Magister Ilmu Kedokteran Klinis,LAM-PTKes,1,1,75,72.0,20.0%,-31.500000,Lolos,60.0,Lebih dari 1 Tahun ke Depan
3,Magister Ilmu Kedokteran Tropis,LAM-PTKes,1,1,14,15.0,20.0%,-33.174603,Lolos,11.0,Lebih dari 1 Tahun ke Depan
4,Magister Ilmu Pendidikan Kedokteran dan Kesehatan,LAM-PTKes,1,1,5,7.0,20.0%,-36.190476,Lolos,4.0,Lebih dari 1 Tahun ke Depan
...,...,...,...,...,...,...,...,...,...,...,...
143,Magister Pengkajian Amerika,BAN-PT,1,1,24,25.0,20.0%,-33.277778,Lolos,19.0,Lebih dari 1 Tahun ke Depan
144,Magister Kajian Budaya Timur Tengah,BAN-PT,1,1,0,2.0,20.0%,inf,Lolos,0.0,Lebih dari 1 Tahun ke Depan
145,Magister Kajian Budaya dan Media,BAN-PT,1,1,41,40.0,20.0%,-31.686992,Lolos,32.0,Lebih dari 1 Tahun ke Depan
146,Magister Ketahanan Nasional,BAN-PT,1,1,70,67.0,20.0%,-31.364606,Lolos,56.0,Lebih dari 1 Tahun ke Depan
